# NEC winter-school: third exercise

In this notebook, we will learn how to finetune a model. To do that, we will take the model that we previously trained in MNIST and finetune it in Fashion-MNIST.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np                     # Basic array operation library
import matplotlib.pyplot as plt        # For plotting and displaying
%matplotlib inline

Copy your class model, but replace your last layer.

In [8]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(2, activation='relu')
        self.d3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

    def get_emb(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


In [9]:
# Create an instance of the model
model = MyModel()
model.build((None,28,28,1))


In [12]:
model.load_weights('mnist_model.h5')

In [13]:
model.compile(optimizer='adam',                         # Optimizer
              loss='sparse_categorical_crossentropy',   # Loss function
              metrics=['accuracy'])                     # Test metric to control that everything is working well